# Header

In [5]:
from time import time

# Data manipulation
import pandas as pd

# Metrics 
from sklearn.metrics import accuracy_score, f1_score

In [6]:
from data_treatment import train_df, test_df, \
                            treated_train_df, treated_test_df, \
                            new_features_train_df, new_features_test_df, \
                            targets_for_test_df

from data_treatment import train_val_split                            

In [7]:



datetime_train_df = train_df.copy()
datetime_test_df = test_df.copy()
datetime_treated_train_df = treated_train_df.copy()
datetime_treated_test_df = treated_test_df.copy()
datetime_new_features_train_df = new_features_train_df.copy()
datetime_new_features_test_df = new_features_test_df.copy()

# Ensure that 'timestamp' is in datetime format and set as the index for all dataframes
datetime_train_df['timestamp'] = pd.to_datetime(datetime_train_df['timestamp'])
datetime_test_df['timestamp'] = pd.to_datetime(datetime_test_df['timestamp'])
datetime_treated_train_df['timestamp'] = pd.to_datetime(datetime_treated_train_df['timestamp'])
datetime_treated_test_df['timestamp'] = pd.to_datetime(datetime_treated_test_df['timestamp'])
datetime_new_features_train_df['timestamp'] = pd.to_datetime(datetime_train_df['timestamp'])
datetime_new_features_test_df['timestamp'] = pd.to_datetime(datetime_test_df['timestamp'])

# Set 'timestamp' as the index
datetime_train_df = datetime_train_df.set_index('timestamp')
datetime_test_df = datetime_test_df.set_index('timestamp')
datetime_treated_train_df = datetime_treated_train_df.set_index('timestamp')
datetime_treated_test_df = datetime_treated_test_df.set_index('timestamp')
datetime_new_features_train_df = datetime_new_features_train_df.set_index('timestamp')
datetime_new_features_test_df = datetime_new_features_test_df.set_index('timestamp')

# Arima for close price

## Growth factor

In [8]:
from statsmodels.tsa.arima.model import ARIMA

start_time = time()
X_train, y_train, X_val, y_val = train_val_split(train_df)
close_train_series = X_train['close']
close_val_series = X_val['close']

close_train_growth = close_train_series.pct_change()
close_val_growth = close_val_series.pct_change()

close_train_growth = close_train_growth.dropna()
close_val_growth = close_val_growth.dropna()

# Represent growth as integers to save space
close_train_growth = (close_train_growth * 1000).astype('int32')
close_val_growth = (close_val_growth * 1000).astype('int32')

# Fit ARIMA model on close_train_growth
arima_model = ARIMA(close_train_growth, order=(5, 1, 0))
arima_result = arima_model.fit()

print(arima_result.summary())

end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')

C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, forecasts cannot be generated. To use the model for forecasting, use one of the supported classes of index.
  self._init_dates(dates, freq)
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:473: ValueWarning: An unsupported index was provided. As a result, 

MemoryError: Unable to allocate 466. MiB for an array with shape (6, 6, 1697950) and data type float64

### Forecast

In [ ]:
# Forecast the growth prices using the ARIMA model
forecast_steps = len(close_val_growth)
forecasted_growth = arima_result.forecast(steps=forecast_steps)

forecasted_target = forecasted_growth >= 0


C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


1697949   -0.000787
1697950   -0.001069
1697951   -0.000913
1697952   -0.000772
1697953   -0.000831
             ...   
2122431   -0.000832
2122432   -0.000832
2122433   -0.000832
2122434   -0.000832
2122435   -0.000832
Name: predicted_mean, Length: 424487, dtype: float64


In [ ]:
close_test_series = test_df['close']
close_test_growth = close_test_series.pct_change()

# Forecast the growth prices using the ARIMA model
forecast_steps = len(close_test_growth)
forecasted_growth = arima_result.forecast(steps=forecast_steps)

forecasted_target = forecasted_growth >= 0


C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(


1697949   -0.000787
1697950   -0.001069
1697951   -0.000913
1697952   -0.000772
1697953   -0.000831
             ...   
2607561   -0.000832
2607562   -0.000832
2607563   -0.000832
2607564   -0.000832
2607565   -0.000832
Name: predicted_mean, Length: 909617, dtype: float64


In [ ]:

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, forecasted_target[:len(targets_for_test_df)])	
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, forecasted_target[:len(targets_for_test_df)], average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

Test Accuracy: 0.58005
Test F1 Macro Score: 0.36711


### Prediction (rolling forecast)

In [ ]:
# Initialize a list to store the predictions
predicted_growth = []

# Use the last observed value from the training set as the initial input
input_series = close_train_growth.copy()

# Convert

# Iteratively predict the next value in the validation set
for actual_value in close_val_growth:

    # print(f'iteration number: {len(predicted_growth)}')
    # print(input_series)

    # Forecast the next value
    next_forecast = float(arima_result.forecast(steps=1).iloc[0])
    predicted_growth.append(next_forecast)

    # Update the input series with the actual value from the validation set
    input_series = pd.concat([input_series, pd.Series([actual_value])], ignore_index=True)
    # Ensure that 'timestamp' is in datetime format and set as the index

    # if len(predicted_growth) > 5:
    #     break


predicted_target = [growth >= 0 for growth in predicted_growth]
# Print the predicted growth values
print(predicted_growth)
print(predicted_target)


iteration number: 0
0          0.000721
1         -0.000621
2          0.000000
3         -0.000100
4         -0.001431
             ...   
1697944   -0.001913
1697945   -0.001757
1697946   -0.000480
1697947   -0.001280
1697948   -0.000160
Name: close, Length: 1697949, dtype: float64
iteration number: 1
0          0.000721
1         -0.000621
2          0.000000
3         -0.000100
4         -0.001431
             ...   
1697945   -0.001757
1697946   -0.000480
1697947   -0.001280
1697948   -0.000160
1697949   -0.001446
Length: 1697950, dtype: float64


C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: FutureWarning: No supported index is available. In the next version, calling this method in a model without a supported index will result in an exception.
  return get_prediction_index(
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
 

iteration number: 2
0          0.000721
1         -0.000621
2          0.000000
3         -0.000100
4         -0.001431
             ...   
1697946   -0.000480
1697947   -0.001280
1697948   -0.000160
1697949   -0.001446
1697950   -0.000161
Length: 1697951, dtype: float64
iteration number: 3
0          0.000721
1         -0.000621
2          0.000000
3         -0.000100
4         -0.001431
             ...   
1697947   -0.001280
1697948   -0.000160
1697949   -0.001446
1697950   -0.000161
1697951    0.000000
Length: 1697952, dtype: float64
iteration number: 4
0          0.000721
1         -0.000621
2          0.000000
3         -0.000100
4         -0.001431
             ...   
1697948   -0.000160
1697949   -0.001446
1697950   -0.000161
1697951    0.000000
1697952    0.002575
Length: 1697953, dtype: float64


C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(
C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


iteration number: 5
0          0.000721
1         -0.000621
2          0.000000
3         -0.000100
4         -0.001431
             ...   
1697949   -0.001446
1697950   -0.000161
1697951    0.000000
1697952    0.002575
1697953   -0.002568
Length: 1697954, dtype: float64
[-0.0007866790288001808, -0.0007866790288001808, -0.0007866790288001808, -0.0007866790288001808, -0.0007866790288001808, -0.0007866790288001808]
[False, False, False, False, False, False]


C:\Users\PLour\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\LocalCache\local-packages\Python311\site-packages\statsmodels\tsa\base\tsa_model.py:837: ValueWarning: No supported index is available. Prediction results will be given with an integer index beginning at `start`.
  return get_prediction_index(


In [ ]:

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)])	
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)], average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

In [ ]:
close_test_series = test_df['close']
close_test_growth = close_test_series.pct_change()

# Forecast the growth prices using the ARIMA model
forecast_steps = len(close_test_growth)
forecasted_growth = arima_result.forecast(steps=forecast_steps)

forecasted_target = forecasted_growth >= 0

# Print the forecasted growth prices
print(forecasted_growth)

In [ ]:

# Calculate accuracy
accuracy = accuracy_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)])	
print(f'Test Accuracy: {accuracy:.5f}')

# Calculate F1 macro score
f1_macro = f1_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)], average='macro')
print(f'Test F1 Macro Score: {f1_macro:.5f}')

## Price Diff

In [ ]:
from statsmodels.tsa.arima.model import ARIMA

start_time = time()
X_train, y_train, X_val, y_val = train_val_split(train_df)
close_train_series = X_train['close']
close_val_series = X_val['close']

# Get Close difference
close_train_diff = close_train_series.diff().dropna()
close_val_diff = close_val_series.diff().dropna()

# Represent as integers to save space
close_train_diff = (close_train_diff * 1000).astype('int32')
close_val_diff = (close_val_diff * 1000).astype('int32')


# Fit ARIMA model on close_train_growth
arima_model = ARIMA(close_train_diff, order=(5, 1, 0))
arima_result = arima_model.fit()

print(arima_result.summary())

end_time = time.time()
minutes = (end_time - start_time) // 60
seconds = (end_time - start_time) % 60
print(f'Time elapsed: {minutes:.0f}m {seconds:.1f}s')
print('--------------------------------------')

### Forecast

In [ ]:
# Forecast the diff prices using the ARIMA model
forecast_steps = len(close_val_diff)
forecasted_diff = arima_result.forecast(steps=forecast_steps)

forecasted_target = forecasted_diff >= 0

# Print the forecasted diff prices
print(forecasted_diff)

In [ ]:

# # Calculate accuracy
# accuracy = accuracy_score(targets_for_test_df, forecasted_target[:len(targets_for_test_df)])	
# print(f'Test Accuracy: {accuracy:.5f}')

# # Calculate F1 macro score
# f1_macro = f1_score(targets_for_test_df, forecasted_target[:len(targets_for_test_df)], average='macro')
# print(f'Test F1 Macro Score: {f1_macro:.5f}')

### Prediction (rolling forecast)

In [ ]:
# Initialize a list to store the predictions
predicted_diff = []

# Use the last observed value from the training set as the initial input
input_series = close_train_diff.copy()

# Iteratively predict the next value in the validation set
for actual_value in close_val_diff:
    
    # Forecast the next value
    next_forecast = arima_result.forecast(steps=1)[0]
    predicted_diff.append(next_forecast)
    
    # Update the input series with the actual value from the validation set
    input_series = input_series.append(pd.Series([actual_value]))

predicted_target = predicted_diff >= 0
# Print the predicted diff values
print(predicted_diff)
print(predicted_target)


In [ ]:

# # Calculate accuracy
# accuracy = accuracy_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)])	
# print(f'Test Accuracy: {accuracy:.5f}')

# # Calculate F1 macro score
# f1_macro = f1_score(targets_for_test_df, predicted_target[:len(targets_for_test_df)], average='macro')
# print(f'Test F1 Macro Score: {f1_macro:.5f}')